# Athena Query Examples

The following example demonstrates the Jupyter IR Tools Athena module, which simplifies the task of running Athena queries in a Jupyter notebook to minimize the code clutter needed to run a notebook.

If you use SSO to authenticate the notebook, run the following cell and replace the account_id with the account_id that holds the Glue data catalog, and replace Jupyter-IR-AdministratorAccess with the SSO permission set that has access to the AWS account and Athena permissions.

If you are using an instance profile or IAM user, you can delete the next cell.

In [ ]:
# Only use this if you are using SSO to authenticate the notebook
from jupyterirtools import jupyterauth, athena


## Methods
**run_query**: This method takes two paramaters
* **sql**: A python string of the SQL statement to be executed. You can include python "f-string" style variables to make the SQL more dynamic. By passing in a variable surrounded by brackets, the value of the variable will be substituted inside the brackets. Example: 'Select * from {table}' will substitute the value for the variable table in the params dictionary.
* **params**: (Optional) A dictionary object with the variables that are replaced in the SQL string.
* **returns**: A two-element array with the Pandas dataframe and the sql statement executed.

**run_named_query_with_sql**: This method takes two paramaters
* **source**: The source of the named query. This will be a suffix on the query name to permit different methods of accessing the data such as LakeFormation or native CloudTrail.
* **Query Name**: The name of the named query to retrieve.
* **params**: (Optional) A dictionary object with the variables that are replaced in the SQL string.
* **returns**: A two-element array with the Pandas dataframe and the sql statement executed.

**run_named**: This method takes two paramaters
* **source**: The source of the named query. This will be a suffix on the query name to permit different methods of accessing the data such as LakeFormation or Natic CloudTrail.
* **Query Name**: The name of the named query to retrieve.
* **params**: (Optional) A dictionary object with the variables that are replaced in the SQL string.
* **returns**: The Pandas dataframe of the query results.

## Athena Named Queries with Parameters

This library supports Athena queries with parameters. The Athena query syntax is a simple question mark ? placeholder for the parameter in the query, and the parameters are passed into Athena with an array. This leads to very hard to read and brittle code in the notebook, so the library takes in the parameters as a dictionary of name value pairs.

The query then has comments at the top that list the parameter to dictionary mapping. The comment starts the line with `-- PARAM` followed by the dictionary value name, then an option python formatting string. In the case below, there are 3 parameters, starttime, endtime, and accountid. The starttime and endtime are formtted t be YYYYMMDDHH. 

#### Athena Named Query
```SQL
-- PARAM starttime %Y%d%m%H
-- PARAM endtime %Y%d%m%H
-- PARAM accountid
SELECT * 
FROM tablename
WHERE timestamp >= ?
AND timestamp <= ?
AND accountid = ?
```

*There is an issue with the Athena console that doesn't recognize parameters after a comment. To run the query in the console, you will need to delete the comments. It is best to copy the query into a new window without the comments to avoid overwriting. You can place the PARAM comments at the end of the query and it will work fine in both Jupyter and Athena.*


See the example below to see how to execute the named query.

In [ ]:
from datetime import datetime, timedelta
import importlib
importlib.reload(athena)

source="security_lake" # Would be security_lake or athena

df, sql, params = athena.run_named_query_with_sql_params(source, 'cloudtrail', {
    "starttime": datetime.now() - timedelta(hours = 1),
    "endtime": datetime.now(),
    "account": "0123456789012"
    })

print(sql)
print(params)
df

In [ ]:
from jupyterirtools import athena

df, sql = athena.run_query_sql('''
SELECT * FROM "amazon_security_lake_glue_db_us_east_1"."amazon_security_lake_table_us_east_1_cloud_trail" limit 10;
''')
print(sql)
df

In [ ]:
from datetime import datetime, timedelta
import importlib
importlib.reload(athena)

source="security_lake" # Would be security_lake or athena

df, sql = athena.run_named_query_with_sql(source, 'cloudtrail', {
    "starttime": datetime.now() - timedelta(hours = 1),
    "endtime": datetime.now()
    })

print(sql)
df

In [ ]:
df = athena.run_named_query(source, 'cloudtrail2')

df

In [ ]:
df, sql = athena.run_query_sql("""
SELECT * 
FROM amazon_security_lake_glue_db_us_east_1.amazon_security_lake_table_us_east_1_cloud_trail
WHERE accountid = '{accountid}' limit 10;""")
print(sql)
df
